In [1]:
import numpy as np
import argparse
import random
import time
import os
import warnings
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import rpy2.robjects as robjects
import matplotlib.pyplot as plt
import copy
import torchvision

from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from rpy2.robjects.packages import importr
from torch.optim.optimizer import Optimizer, required
from torch.optim.lr_scheduler import StepLR

%matplotlib inline
warnings.filterwarnings('ignore')

## Install R package BayesGPfit

utils = importr('utils')
utils.install_packages('BayesGPfit', verbose = 0)
GP = importr('BayesGPfit')

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/BayesGPfit_0.1.0.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 26941 bytes (26 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[wri

## 1. Model

### 1.1  BNNSTGP

In [2]:
## BNNSTGP Model

class BNNSTGP_two_layer(nn.Module):
    def __init__(self, input_dim, n_hid, n_hid2, output_dim, w_dim, n_knots, phi, lamb=1.,
                 b_prior_sig=1, zeta_prior_sig=1, eta_prior_sig=1, alpha_prior_sig=1,
                 act='relu'):
        super(BNNSTGP_two_layer, self).__init__()

        self.input_dim = input_dim
        self.n_hid = n_hid
        self.n_hid2 = n_hid2
        self.output_dim = output_dim
        self.w_dim = w_dim
        self.n_knots = n_knots
        self.phi = phi  
        self.lamb = lamb

        self.sigma = nn.Parameter(torch.tensor(1.))
        self.b = nn.Parameter(torch.Tensor(n_knots, n_hid).normal_(0, 1))
        self.zeta = nn.Parameter(torch.Tensor(n_hid2, output_dim).uniform_(-1, 1))
        self.eta = nn.Parameter(torch.Tensor(n_hid).zero_())
        self.alpha = nn.Parameter(torch.Tensor(w_dim, output_dim).zero_())
        self.fc = nn.Linear(n_hid, n_hid2)

        self.b_prior_sig = b_prior_sig
        self.zeta_prior_sig = zeta_prior_sig
        self.eta_prior_sig = eta_prior_sig
        self.alpha_prior_sig = alpha_prior_sig

        if act == 'relu':
            self.act = torch.relu
        elif act == 'tanh':
            self.act = torch.tanh
        elif act == 'sigmoid':
            self.act = torch.sigmoid
        else:
            raise ValueError('Invalid activation function %s' % act)

    def forward(self, x, w):
        out = torch.mm(self.phi, self.b)  ## beta_tilde, (p, n_hid)
        out = F.threshold(out, self.lamb, self.lamb) - F.threshold(-out, self.lamb, self.lamb)  ## g_lambda
        out = self.sigma * out
        out = torch.mm(x, out) + self.eta  ## beta, (N, n_hid)

        out = self.act(out)
        out = self.fc(out)
        out = self.act(out)
        out = torch.mm(out, self.zeta)  # + torch.mm(w, self.alpha)

        return out

    def log_prior(self):
        logprior = 0.5 * (self.b ** 2).sum() / (self.b_prior_sig ** 2)
        logprior += 0.5 * (self.zeta ** 2).sum() / (self.zeta_prior_sig ** 2)
        logprior += 0.5 * (self.eta ** 2).sum() / (self.eta_prior_sig ** 2)
        logprior += 0.5 * (self.alpha ** 2).sum() / (self.alpha_prior_sig ** 2)
        return logprior




### 1.2 SGLD 

In [3]:


## SGLD (Stochastic Gradient Langevin Dynamics)

class SGLD(Optimizer):
    def __init__(self, params, lr = required, langevin = True):
        self.langevin = langevin
        defaults = dict(lr=lr)
        super(SGLD, self).__init__(params, defaults)

    @torch.no_grad()
    def step(self):
        loss = None
        
        for group in self.param_groups:
            
            for p in group['params']:
                if p.grad is None:
                    continue
                d_p = p.grad
                
                if self.langevin == True:
                    langevin_noise = p.new(p.size()).normal_(mean=0, std=1)/np.sqrt(group['lr'])
                    p.add_(0.5*d_p + langevin_noise, alpha = -group['lr'])

                else:
                    p.add_(0.5*d_p, alpha = -group['lr'])


        return loss


### 1.3 Network Wrapper

In [4]:

## Network Wrapper
class Net(object):

    def __init__(self, task='binary', lr=1e-3, input_dim=784, n_hid = 128, n_hid2 = 64, output_dim = 1, w_dim = 1, n_knots = 66,
                 N_train=200, phi=None, lamb = 1, langevin = True, step_decay_epoch = 100, step_gamma = 0.1, act = 'relu'):

        # print(' Creating Net!! ')
        self.task = task
        if task not in ['binary', 'multiclass']:
            raise ValueError('Invalid task %s' % task)
        self.lr = lr

        self.input_dim = input_dim
        self.n_hid = n_hid
        self.n_hid2 = n_hid2
        self.output_dim = output_dim
        self.w_dim = w_dim
        self.n_knots = n_knots
        self.phi = phi
        self.lamb = lamb
        self.act = act

        self.N_train = N_train
        self.langevin = langevin
        self.step_decay_epoch = step_decay_epoch
        self.step_gamma = step_gamma

        self.create_net()
        self.create_opt()
        self.epoch = 0

        self.weight_set_samples = []


    def create_net(self):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model = BNNSTGP_two_layer(input_dim=self.input_dim, n_hid=self.n_hid, n_hid2=self.n_hid2, output_dim=self.output_dim,
                                       w_dim=self.w_dim, n_knots = self.n_knots, phi=torch.tensor(self.phi).to(self.device),
                                       lamb = self.lamb, act = self.act)
        self.model.to(self.device)
        # print('    Total params: %.2fK' % (self.get_nb_parameters() / 1000.0))


    def create_opt(self):
        self.optimizer = SGLD(params=self.model.parameters(), lr=self.lr, langevin = self.langevin)
        self.scheduler = StepLR(self.optimizer, step_size = self.step_decay_epoch, gamma=self.step_gamma)


    def fit(self, x, w, y, threshold=0.5):
        x = x.to(self.device)
        w = w.to(self.device)

        if self.task == 'binary':
            y = y.float().to(self.device).reshape(-1, 1)
            self.optimizer.zero_grad()

            out = self.model(x, w)
            loss = F.binary_cross_entropy_with_logits(out, y, reduction='mean')
            loss = loss * self.N_train
            # loss += self.model.log_prior()

            loss.backward()
            self.optimizer.step()

            pred = (torch.sigmoid(out ) >threshold).long()
            accu = (pred == y.long()).sum().float()

        else:                           ## multiclass
            y = y.long().to(self.device).reshape(-1)
            self.optimizer.zero_grad()

            out = self.model(x, w)
            loss = F.cross_entropy(out, y, reduction = 'mean')
            loss = loss * self.N_train
            loss += self.model.log_prior()

            loss.backward()
            self.optimizer.step()

            pred = out.max(dim=1, keepdim=False)[1]
            accu = (pred == y).sum()

        return loss *x.shape[0 ] /self.N_train, accu


    def eval(self, x, w, y, threshold=0.5):
        x = x.to(self.device)
        w = w.to(self.device)

        if self.task == 'binary':
            y = y.float().to(self.device).reshape(-1, 1)

            out = self.model(x, w)
            loss = F.binary_cross_entropy_with_logits(out, y, reduction='mean')
            loss = loss * self.N_train
            loss += self.model.log_prior()

            pred = (torch.sigmoid(out ) >threshold).float()
            accu = (pred == y).sum().float()

        else:                        ## multiclass
            y = y.long().to(self.device).reshape(-1)

            out = self.model(x, w)
            loss = F.cross_entropy(out, y, reduction = 'mean')
            loss = loss * self.N_train
            loss += self.model.log_prior()

            pred = out.max(dim=1, keepdim=False)[1]
            accu = (pred == y).sum()

        return loss *x.shape[0 ] /self.N_train, accu


    def get_nb_parameters(self):
        return sum(p.numel() for p in self.model.parameters())


    def save_net_weights(self, max_samples):

        if len(self.weight_set_samples) >= max_samples:
            self.weight_set_samples.pop(0)

        self.weight_set_samples.append(copy.deepcopy(self.model.state_dict()))
        # print(' saving weight samples %d/%d' % (len(self.weight_set_samples), max_samples) )


    def all_sample_eval(self, x, w, y, threshold=0.5):
        x = x.to(self.device)
        w = w.to(self.device)
        y = y.float().to(self.device)

        pred = x.new(len(self.weight_set_samples), x.shape[0], self.output_dim)

        for i, weight_dict in enumerate(self.weight_set_samples):
            self.model.load_state_dict(weight_dict)
            out_i = self.model(x, w)
            pred[i] = torch.sigmoid(out_i)

        pred = (pred.mean(0 ) >threshold).float()
        accu = (pred == y).sum().float()

        return accu


    def save(self, filename):
        print('Writting %s\n' % filename)
        torch.save({
            'epoch': self.epoch,
            'lr': self.lr,
            'model': self.model,
            'optimizer': self.optimizer,
            'scheduler': self.scheduler}, filename)


    def load(self, filename):
        print('Reading %s\n' % filename)
        state_dict = torch.load(filename)
        self.epoch = state_dict['epoch']
        self.lr = state_dict['lr']
        self.model = state_dict['model']
        self.optimizer = state_dict['optimizer']
        self.scheduler = state_dict['scheduler']
        print('  restoring epoch: %d, lr: %f' % (self.epoch, self.lr))
        return self.epoch



## 2. Load Fashion MNIST Data

In [5]:

transform = transforms.Compose([
    transforms.ToTensor(),
])

f_train = torchvision.datasets.FashionMNIST(root = './data/fmnist', train = True, download = True, transform = transform)
f_val = torchvision.datasets.FashionMNIST(root = './data/fmnist', train = False, download = True, transform = transform)


In [6]:
robjects.r['load']("./index/fmnist_idx.RData")

class mydata(Dataset):
    def __init__(self, x_list, y_list):
        self.x_list = x_list
        self.y_list = y_list
    def __len__(self):
        return len(self.y_list)
    def __getitem__(self, i):
        x = self.x_list[i].reshape(-1)
        w = torch.tensor([1.])
        y = torch.tensor([self.y_list[i]])
        return (x, w), y

## 3. Training using full data

In [7]:

def FMNIST_all(c1=3, c2=5, n_epochs=10000, lr=1e-5, lamb=2.5, b=10, langevin=True, seed=17, act='relu'):

    torch.manual_seed(seed)
    random.seed(seed)

    x_list_c1 = []
    y_list_c1 = []
    x_list_c2 = []
    y_list_c2 = []

    for x, y in f_train:
        if y == c1:
            x_list_c1.append(x)
            y_list_c1.append(0)
        if y == c2:
            x_list_c2.append(x)
            y_list_c2.append(1)

    n = 5000
    x_list = random.sample(x_list_c1, n) + random.sample(x_list_c2, n)
    y_list = random.sample(y_list_c1, n) + random.sample(y_list_c2, n)

    FashionMNIST_2_train = mydata(x_list, y_list)

    ## test set

    x_list = []
    y_list = []

    for x, y in f_val:
        if y == c1:
            x_list.append(x)
            y_list.append(0)
        if y == c2:
            x_list.append(x)
            y_list.append(1)

    FashionMNIST_2_test = mydata(x_list, y_list)

    n_train = len(FashionMNIST_2_train)
    n_test = len(FashionMNIST_2_test)

    train_batch_size = 128
    test_batch_size = 128

    train_loader = DataLoader(FashionMNIST_2_train, batch_size=train_batch_size, shuffle=True)
    val_loader = DataLoader(FashionMNIST_2_test, batch_size=test_batch_size, shuffle=True)

    grids = GP.GP_generate_grids(d=2, num_grids=28)
    phi = GP.GP_eigen_funcs_fast(grids, b=b, poly_degree=30)
    phi = np.array(phi)

    torch.set_default_dtype(torch.float32)


    net = Net(lr=lr, input_dim=784, n_hid=128, n_hid2=32, output_dim=1, w_dim=1, n_knots=phi.shape[1],
              N_train=2 * n, phi=torch.tensor(phi, dtype=torch.float32), lamb=lamb, langevin=langevin,
              step_decay_epoch=2000, step_gamma=0.2, act=act)

    epoch = 0

    start_save = 3 * n_epochs / 4
    save_every = 2
    N_saves = 100
    test_every = 20
    print_every = 100

    loss_train = np.zeros(n_epochs)
    accu_train = np.zeros(n_epochs)

    loss_val = np.zeros(n_epochs)
    accu_val = np.zeros(n_epochs)

    best_accu = 0


    for i in range(epoch, n_epochs):

        tic = time.time()
        net.scheduler.step()

        for (x, w), y in train_loader:
            loss, accu = net.fit(x, w, y)
            loss_train[i] += loss
            accu_train[i] += accu

        loss_train[i] /= n_train
        accu_train[i] /= n_train
        toc = time.time()

        if i > start_save and i % save_every == 0:
            net.save_net_weights(max_samples=N_saves)



        if i % test_every == 0:
            with torch.no_grad():
                tic = time.time()
                for (x, w), y in val_loader:
                    loss, accu = net.eval(x, w, y)

                    loss_val[i] += loss
                    accu_val[i] += accu

                loss_val[i] /= n_test
                accu_val[i] /= n_test
                toc = time.time()
                best_accu = max(best_accu, accu_val[i])

            if i % print_every == 0:
                print('Epoch %d, train time %.4f s, train loss %.4f, train accuracy %.2f%%' % (i, toc-tic, loss_train[i], accu_train[i]*100))
                print('  Epoch %d, test time %.4f s, test loss %.4f, test accuracy %.2f%%' % (i, toc-tic, loss_val[i], accu_val[i]*100))
        
    print('%d vs %d, best test accuracy: %s' % (c1, c2, best_accu))
    
    return net


## Sandal (5) vs Sneaker (7)

In [8]:
net = FMNIST_all(c1=5, c2=7, n_epochs = 5000, lr = 1e-6, b = 10, lamb = 2.5, langevin = True)

Epoch 0, train time 0.0657 s, train loss 1.7370, train accuracy 73.57%
  Epoch 0, test time 0.0657 s, test loss 3.5872, test accuracy 76.75%
Epoch 100, train time 0.0620 s, train loss 0.1068, train accuracy 96.27%
  Epoch 100, test time 0.0620 s, test loss 3.3088, test accuracy 96.45%
Epoch 200, train time 0.0618 s, train loss 0.1094, train accuracy 96.08%
  Epoch 200, test time 0.0618 s, test loss 3.4229, test accuracy 92.30%
Epoch 300, train time 0.0608 s, train loss 0.0754, train accuracy 97.29%
  Epoch 300, test time 0.0608 s, test loss 3.3403, test accuracy 96.80%
Epoch 400, train time 0.0609 s, train loss 0.0727, train accuracy 97.34%
  Epoch 400, test time 0.0609 s, test loss 3.3867, test accuracy 95.70%
Epoch 500, train time 0.0620 s, train loss 0.0662, train accuracy 97.62%
  Epoch 500, test time 0.0620 s, test loss 3.3953, test accuracy 96.45%
Epoch 600, train time 0.0587 s, train loss 0.0653, train accuracy 97.69%
  Epoch 600, test time 0.0587 s, test loss 3.4153, test accur

## 4. Training using limited data

In [9]:

def FMNIST_50(c1=5, c2=7, size=10,
              b=10., lamb=5., poly_degree=30,
              n_hid=128, n_hid2=64, lr=1e-4, n_epochs=200, langevin=False):

    beta = np.zeros([50, 100, 28 * 28, 128])

    torch.manual_seed(17)
    random.seed(17)

    accus = []

    if size == 10:
        k = 0
    elif size == 20:
        k = 1
    elif size == 30:
        k = 2
    elif size == 40:
        k = 3
    elif size == 50:
        k = 4
    else:
        raise ValueError('Invalid Size')

    if c1 == 0 and c2 == 2:
        dat = 'idx02'
    elif c1 == 0 and c2 == 6:
        dat = 'idx06'
    elif c1 == 5 and c2 == 7:
        dat = 'idx57'
    elif c1 == 7 and c2 == 9:
        dat = 'idx79'
    else:
        raise ValueError('Invalid Pair')

    grids = GP.GP_generate_grids(d=2, num_grids=28)
    phi = GP.GP_eigen_funcs_fast(grids, b=b, poly_degree=poly_degree)
    phi = np.array(phi)

    x_list = []
    y_list = []

    for x, y in f_val:
        if y == c1:
            x_list.append(x)
            y_list.append(0)
        if y == c2:
            x_list.append(x)
            y_list.append(1)

    FashionMNIST_2_test = mydata(x_list, y_list)

    # for id in tqdm(range(100)):
    for id in tqdm(range(50)):
        idx = set(np.array(robjects.r[dat][k], dtype=np.int)[10,])

        ii = 0
        x_list = []
        y_list = []
        for x, y in f_train:
            ii = ii + 1
            if ii in idx:
                x_list.append(x)
                if y == c1:
                    y_list.append(0)
                elif y == c2:
                    y_list.append(1)

        FashionMNIST_2_train = mydata(x_list, y_list)

        n_train = len(FashionMNIST_2_train)
        n_test = len(FashionMNIST_2_test)

        torch.set_default_dtype(torch.float32)

        train_batch_size = min(128, size * 2)
        test_batch_size = min(128, size * 2)

        train_loader = DataLoader(FashionMNIST_2_train, batch_size=train_batch_size, shuffle=True)
        val_loader = DataLoader(FashionMNIST_2_test, batch_size=test_batch_size, shuffle=True)

        net = Net(lr=lr, input_dim=784, n_hid=n_hid, n_hid2=n_hid2, output_dim=1, w_dim=1, n_knots=phi.shape[1],
                  N_train=2 * size, phi=torch.tensor(phi, dtype=torch.float32), lamb=lamb, langevin=langevin,
                  step_decay_epoch=1000, step_gamma=1)

        epoch = 0

        start_save = 4 * n_epochs / 5
        save_every = 2
        N_saves = 100


        loss_train = np.zeros(n_epochs)
        accu_train = np.zeros(n_epochs)

        best_test = 0
        for i in range(epoch, n_epochs):

            for (x, w), y in train_loader:
                loss, accu = net.fit(x, w, y)
                net.scheduler.step()
                loss_train[i] += loss
                accu_train[i] += accu


            loss_train[i] /= n_train
            accu_train[i] /= n_train

            if i > start_save and i % save_every == 0:
                net.save_net_weights(max_samples=N_saves)
            if i % 100 == 0:
                accu_test = 0
                with torch.no_grad():

                    for (x, w), y in val_loader:
                        _, accu = net.eval(x, w, y)
                        accu_test += accu
                    accu_test /= n_test
                    best_test = max(best_test, accu_test)

        accus.append(best_test.detach().cpu().item())
        print("dataset %d, accuracy= %.4f" % (id, best_test))


        for j, weight_dict in enumerate(net.weight_set_samples):
            net.model.load_state_dict(weight_dict)
            tmp = torch.mm(net.model.phi, net.model.b)
            tmp = F.threshold(tmp, net.model.lamb, net.model.lamb) - F.threshold(-tmp, net.model.lamb, net.model.lamb)
            tmp = net.model.sigma * tmp
            beta[id, j] = tmp.cpu().detach().numpy()

    print('c1=%d, c2=%d, n = %d, accu = %.4f (%.4f)' %(c1, c2, size, np.mean(accus), np.std(accus)))

    return beta


Sandal (5) vs Sneaker (7)

size of data = 2*10 = 20

In [10]:
beta50 = FMNIST_50(c1 = 5, c2 = 7, size = 10,
      b = 10, lamb = 2.5, poly_degree = 30, 
      n_hid = 128, n_hid2 = 64, lr = 1e-4, n_epochs = 10000, langevin = True)

  2%|▏         | 1/50 [01:02<50:58, 62.41s/it]

dataset 0, accuracy= 0.8165


  4%|▍         | 2/50 [02:04<49:53, 62.37s/it]

dataset 1, accuracy= 0.8185


  6%|▌         | 3/50 [03:06<48:46, 62.27s/it]

...

dataset 47, accuracy= 0.8795


 98%|█████████▊| 49/50 [49:32<01:00, 60.26s/it]

dataset 48, accuracy= 0.8440


100%|██████████| 50/50 [50:32<00:00, 60.66s/it]

dataset 49, accuracy= 0.8300
c1=5, c2=7, n = 10, accu = 0.8427 (0.0234)
